In [1]:
from centrex_tlf.hamiltonian import generate_total_reduced_hamiltonian
from centrex_tlf import states, couplings

In [2]:
X_states_approx = states.generate_coupled_states_X(
    states.QuantumSelector(J=[0, 1, 2, 3, 4, 5], electronic=states.ElectronicState.X)
)

B_states_approx = []
for J, F1, F in zip(
    [1, 1, 1, 2, 1, 2, 2, 3, 2, 3, 3, 4],
    [
        1 / 2,
        1 / 2,
        3 / 2,
        3 / 2,
        3 / 2,
        5 / 2,
        3 / 2,
        5 / 2,
        5 / 2,
        7 / 2,
        5 / 2,
        7 / 2,
    ],
    [0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
):
    B_states_approx.extend(
        states.generate_coupled_states_B(
            states.QuantumSelector(
                J=J, F1=F1, F=F, electronic=states.ElectronicState.B, P=[1, -1]
            )
        )
    )

In [3]:
H_total = generate_total_reduced_hamiltonian(
    X_states_approx, B_states_approx, rtol=None, stol=1e-3
)


In [4]:
df = couplings.generate_br_dataframe(
    H_total.X_states, H_total.B_states, group_ground="J", group_excited=True
)
df


,"|B, J = 1, F1 = 1/2, F = 0>","|B, J = 1, F1 = 1/2, F = 1>","|B, J = 1, F1 = 3/2, F = 1>","|B, J = 1, F1 = 3/2, F = 2>","|B, J = 2, F1 = 3/2, F = 1>","|B, J = 2, F1 = 3/2, F = 2>","|B, J = 2, F1 = 5/2, F = 2>","|B, J = 2, F1 = 5/2, F = 3>","|B, J = 3, F1 = 5/2, F = 2>","|B, J = 3, F1 = 5/2, F = 3>","|B, J = 3, F1 = 7/2, F = 3>","|B, J = 4, F1 = 7/2, F = 3>"
states,,,,,,,,,,,,
"|X, J = 0>",0.333333,0.333227,0.242065,0.239835,0.091374,0.093459,0.000034,0.000000,0.000005,0.000000,0.000000,0.000000
"|X, J = 1>",0.500000,0.499937,0.445365,0.444004,0.354729,0.355927,0.262502,0.261765,0.037557,0.038214,0.000020,0.000002
"|X, J = 2>",0.166667,0.166773,0.257935,0.260156,0.408626,0.406518,0.473272,0.472791,0.312434,0.312885,0.265393,0.020353
"|X, J = 3>",0.000000,0.000063,0.054635,0.055996,0.145271,0.144073,0.237498,0.238233,0.462443,0.461770,0.484256,0.293512
"|X, J = 4>",0.000000,0.000000,0.000000,0.000009,0.000000,0.000023,0.026694,0.027209,0.187561,0.187115,0.234607,0.479647
"|X, J = 5>",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000002,0.000000,0.000016,0.015724,0.206486


# Generate latex for paper

In [26]:
import re
import pandas as pd

df_sort = df[sorted(df.columns, key=lambda x: int(x[-2]))]
df_table = df_sort.T * 2

rel_J_labels = [
    r"$bf_{\tilde{J}\rightarrow J=\tilde{J}-2}$",
    r"$bf_{\tilde{J}\rightarrow J=\tilde{J}-1}$",
    r"$bf_{\tilde{J}\rightarrow J=\tilde{J}}$",
    r"$bf_{\tilde{J}\rightarrow J=\tilde{J}+1}$",
    r"$bf_{\tilde{J}\rightarrow J=\tilde{J}+2}$",
]


def parse_J_from_string(s: str) -> int:
    m = re.search(r"J\s*=\s*(\d+)", s)
    return int(m.group(1)) if m else None


# Build a map: original column name → its integer J_col
column_to_J = {col: parse_J_from_string(col) for col in df_table.columns}
# And the reverse: J_col → original column name (so we can look up by integer J)
J_to_column = {Jcol: col for col, Jcol in column_to_J.items()}


def format_value(x: float) -> str:
    if x == 0 or pd.isna(x):
        return ""
    elif abs(x) < 0.0001:
        return f"{x:.5f}"
    else:
        return f"{x:.4f}"


def to_ket_label(s: str) -> str:
    # strip leading '|' and trailing '>' and wrap in \ket{…}
    inner = s.strip()
    if inner.startswith("|") and inner.endswith(">"):
        inner = inner[1:-1]
    inner = inner.replace("B, ", "").replace("J", "\\tilde{J}")
    # inner = inner.replace("B", "\mathrm{B}").replace("J", "\\tilde{J}")
    return r"$\ket{" + inner + "}$"


new_index = df_table.index.tolist()
new_data = []

for row_label in new_index:
    J_row = parse_J_from_string(row_label)
    one_row = []
    for k in (-2, -1, 0, +1, +2):
        if J_row is None:
            raw = None
        else:
            J_target = J_row + k
            col_name = J_to_column.get(J_target)  # maybe None if out of range
            raw = df_table.at[row_label, col_name] if (col_name is not None) else None

        # If raw is None or equals 0, put blank; otherwise format
        if raw is None or raw == 0:
            one_row.append("")
        else:
            one_row.append(format_value(raw))
    new_data.append(one_row)

df_shifted = pd.DataFrame(data=new_data, index=new_index, columns=rel_J_labels)

df_shifted.index = df_shifted.index.map(to_ket_label)

latex_code = df_shifted.to_latex(
    escape=False,
    column_format="l" + "r" * len(rel_J_labels),
    index=True,
    index_names=False,
)

latex_code = latex_code.replace("F1", "\\tilde{F_1}")
print(latex_code)

\begin{tabular}{lrrrrr}
\toprule
 & $bf_{\tilde{J}\rightarrow J=\tilde{J}-2}$ & $bf_{\tilde{J}\rightarrow J=\tilde{J}-1}$ & $bf_{\tilde{J}\rightarrow J=\tilde{J}}$ & $bf_{\tilde{J}\rightarrow J=\tilde{J}+1}$ & $bf_{\tilde{J}\rightarrow J=\tilde{J}+2}$ \\
\midrule
$\ket{\tilde{J} = 1, \tilde{F_1} = 1/2, F = 0}$ &  & 0.6667 & 1.0000 & 0.3333 &  \\
$\ket{\tilde{J} = 1, \tilde{F_1} = 1/2, F = 1}$ &  & 0.6665 & 0.9999 & 0.3335 & 0.0001 \\
$\ket{\tilde{J} = 1, \tilde{F_1} = 3/2, F = 1}$ &  & 0.4841 & 0.8907 & 0.5159 & 0.1093 \\
$\ket{\tilde{J} = 2, \tilde{F_1} = 3/2, F = 1}$ & 0.1827 & 0.7095 & 0.8173 & 0.2905 &  \\
$\ket{\tilde{J} = 1, \tilde{F_1} = 3/2, F = 2}$ &  & 0.4797 & 0.8880 & 0.5203 & 0.1120 \\
$\ket{\tilde{J} = 2, \tilde{F_1} = 3/2, F = 2}$ & 0.1869 & 0.7119 & 0.8130 & 0.2881 & 0.00005 \\
$\ket{\tilde{J} = 2, \tilde{F_1} = 5/2, F = 2}$ & 0.00007 & 0.5250 & 0.9465 & 0.4750 & 0.0534 \\
$\ket{\tilde{J} = 3, \tilde{F_1} = 5/2, F = 2}$ & 0.0751 & 0.6249 & 0.9249 & 0.3751 &  \\
$\ket{\t